In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

## NAEC Workshop
### José Moran, Michael Benzaquen


Take a system of $N$, with each agent $i=1\ldots N$ has a binary choice $S_i=\pm1$, with the positive choice indicating for example that he wants to buy an asset. We suppose now that the agent wants to maximize an "utility function" given by:

$$
U_i = h_i S_i + F S_i + S_i\sum_{j(\neq i)}J_{ij}S_j
$$
where $h_i$ is an idiosyncratic bias proper to each agent. We suppose that the $h_i$ are random iid. variables with a density $\rho(h)$. The agent's decision is then naturally given by:

\begin{equation}\label{eq:decision}
    S_i = \mbox{sign}\left(h_i + F  + \sum_{j(\neq i)}J_{ij}S_j\right):=\mbox{sign}(p_i)
\end{equation}
where $p_i$ is the "local polarization field".

In [ ]:
class RFIM(object):
    '''A class for simulating the Random Field Ising Model '''
    
    def __init__(self, J, F, N, delta):
        '''Create an instance of the RFIM with parameters J,F
        of size N, with fields h with a random Laplace distribution of parameter delta'''
        self.J = J
        self.F = F
        self.N = N
        self.S = np.random.choice([-1,1], size = N)
        self.h = np.random.laplace(scale = delta, size = N)
        self.delta = delta
        
        
    def m(self):
        '''Computes average opinion '''
        return np.mean(self.S)
                                   
    
    
    def p(self, i):
        '''Computes the local opinion polarization for agent i p_i
        inputs: agent label i
        returns: the field p_i'''
        local_m = (self.S.sum() - self.S[i]) / (self.N - 1)
        return self.h[i] + self.F + self.J * local_m
    
    def flip(self,i):
        '''Changes S_i -> -S_i according to the dynamic rule
        input: agent label i
        returns: 0 if the agent didn't change opinion
                 1 if the agent changed opinion
        '''
        should_flip = self.S[i] * np.sign(self.p(i))
        if should_flip == -1:
            self.S[i] *= -1
            return 1
        else:
            return 0
    
    
    def sweep(self):
        '''Goes through all N agents and tries to flip them
        input: none
        returns: number of agents that changed their mind
        '''
        s = 0
        for i in range(N):
            s += self.flip(i)
        return s
    
    def equilibrate(self):
        '''Find the equilibrium of the system'''
        flips = self.sweep()
        while flips > 0:
            flips = self.sweep()
            
            


In [ ]:
delta = 1.2
N = 500
J = 1
F = 15

In [ ]:
sim = RFIM(J,F,N, delta = 1.2) #high heterogeneity
sim2 = RFIM(J,F,N, delta = 0.1) #low heterogeneity, path F:-5->5
sim3 = RFIM(J,F,N, delta=0.1) # low heterogeneity, path F:5-> -5

In [ ]:
Fs = np.linspace(-5,5,100)

#values of m for the different paths
ms = np.empty(len(Fs))
ms2 = np.empty(len(Fs))
ms3 = np.empty(len(Fs))
for i in range(len(Fs)):
    sim.F = Fs[i]
    sim2.F = Fs[i]
    sim3.F = Fs[::-1][i]
    sim.equilibrate()
    sim2.equilibrate()
    sim3.equilibrate()
    ms[i] = sim.m()
    ms2[i] = sim2.m()
    ms3[i] = sim3.m()

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(Fs,ms, label = r'F path $-5\to 5$ and backward, $\Delta=1.2$')
plt.plot(Fs,ms2, label= r'F path $-5\to 5$, $\Delta = 0.1$')
plt.plot(Fs[::-1],ms3, label=r'F path $5 \to -5$, $\Delta = 0.1$')
plt.xlabel(r'$F$', fontsize=18)
plt.ylabel(r'$m$', fontsize =18)
plt.legend(fontsize = 16)

In [ ]:
n_trials = int(5e3)
avalanches = np.empty(n_trials)
#shocks to external field
shocks = np.random.rand(n_trials)
av_sim = RFIM(J,F,N = 1000, delta = 0.1)
#prepare the system close to the critical point
av_sim.F = -0.6
av_sim.S = np.ones(1000)
av_sim.equilibrate()
S = np.copy(av_sim.S)
for i in tqdm(range(n_trials)):
    #set the system again close to tipping point
    av_sim.F = -0.6
    av_sim.S = np.copy(S)
    #shock the external field and get the avalanche size
    av_sim.F *= (1+0.5*shocks[i])
    avalanches[i] = av_sim.sweep()

In [ ]:
plt.figure(figsize = (10,10))
plt.hist(avalanches, cumulative = -1, bins = 500, histtype='step', density=True, label='Avalanche sf')
ns = np.logspace(0,2.5,20)
plt.plot(ns,ns**(-1/2), label=r'Theoretical power law $P_>(s)\sim s^{-1/2}$')
plt.xscale('log')
plt.yscale('log')
plt.legend(fontsize = 18, loc = 'lower left')
plt.xlabel(r'Avalanche size $s$', fontsize = 18)
plt.ylabel(r'Survival function $P_>(s)$', fontsize = 18)
plt.ylim(2e-2,1.2)